In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('F://Projects_ML//Chatbot//intents.json').read()
intents = json.loads(data_file)

Using TensorFlow backend.
C:\anaconda\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\anaconda\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\anaconda\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\anaconda\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Future

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [5]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
for w in words:
 bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1

training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 2.3452 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 0us/step - loss: 2.2960 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 0us/step - loss: 2.4913 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 0us/step - loss: 1.5843 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 0us/step - loss: 1.4154 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 0us/step - loss: 1.1940 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 0us/step - loss: 1.3216 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 0us/step - loss: 0.6454 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 16ms/step - loss: 1.3174 - accuracy: 1.

Epoch 77/200
1/1 [==============================] - 0s 0us/step - loss: 1.8835e-05 - accuracy: 1.0000
Epoch 78/200
1/1 [==============================] - 0s 10ms/step - loss: 1.1444e-05 - accuracy: 1.0000
Epoch 79/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 80/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 81/200
1/1 [==============================] - 0s 8ms/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 82/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 83/200
1/1 [==============================] - 0s 7ms/step - loss: 5.3644e-07 - accuracy: 1.0000
Epoch 84/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 8ms/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 0us/step - loss: 2.3842e-0

Epoch 157/200
1/1 [==============================] - 0s 8ms/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 158/200
1/1 [==============================] - 0s 8ms/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 159/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 160/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 161/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 162/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 163/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 164/200
1/1 [==============================] - 0s 0us/step - loss: 1.0610e-05 - accuracy: 1.0000
Epoch 165/200
1/1 [==============================] - 0s 0us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 166/200
1/1 [==============================] - 0s 0us/step - loss: 